https://medium.com/analytics-vidhya/googlenews-api-live-news-from-google-news-using-python-b50272f0a8f0

```bsah
!pip install GoogleNews
!pip install newspaper3k
!pip install scrapy
```

In [1]:
import pandas as pd

In [2]:
from GoogleNews import GoogleNews

googlenews=GoogleNews(start='12/09/2019',end='05/20/2021')
googlenews.search('COVID-19')

In [3]:
MAX_PAGES = 50
dfs = []

for i in range(MAX_PAGES):
    result=googlenews.page_at(i)
    if len(result) == 0:
        print(f"No more result found after page {i-1}.")
        break
    dfs.append(pd.DataFrame(result))
    if i % 5 == 0:
        print("Page:", i)

Page: 0
Page: 5
Page: 10
Page: 15
Page: 20
Page: 25
Page: 30
'NoneType' object is not iterable
No more result found after page 34.


In [4]:
df_all = pd.concat(dfs).reset_index()
df_all.shape

(348, 8)

In [5]:
df_all.datetime.min()

Timestamp('2020-01-22 00:00:00')

In [6]:
df_all.datetime.max()

Timestamp('2021-05-22 23:25:42.843719')

In [7]:
df_all.head()

,index,title,media,date,datetime,desc,link,img
0,0,FDA and CDC Lift Recommended Pause on Johnson ...,,1 month ago,2021-04-29 23:25:25.687568,Use of the Janssen COVID-19 Vaccine should be ...,https://www.fda.gov/news-events/press-announce...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
1,1,Stephen Karanja: Kenyan anti-vaccine doctor di...,BBC,1 month ago,2021-04-29 23:25:25.690002,A Kenyan doctor who became a vociferous oppone...,https://www.bbc.com/news/world-africa-56922517,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
2,2,Information on the Outbreak of Coronavirus Dis...,,"Feb 13, 2020",2020-02-13 00:00:00.000000,To learn about all aspects of the Massachusett...,https://www.mass.gov/resource/information-on-t...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
3,3,Information on the Outbreak of 2019 Novel Coro...,,"Mar 3, 2020",2020-03-03 00:00:00.000000,"On March 2, DPH announced the first presumptiv...",https://www.mass.gov/2019Coronavirus,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."
4,4,Blueprint for a Safer Economy - Coronavirus CO...,COVID-19.ca.gov,"Jun 15, 2020",2020-06-15 00:00:00.000000,California's blueprint for reopening has crite...,https://covid19.ca.gov/safer-economy/,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////..."


In [9]:
df_all.groupby('media').size()

media
                                                          100
ABC News                                                    1
American Farm Bureau Federation                             1
American Journal of Managed Care                            1
American Veterinary Medical Association                     1
                                                         ... 
Washoe County                                               1
WebMD                                                       3
West Virginia Department of Health and Human Resources      3
World Bank Group                                            1
Yale News                                                   1
Length: 81, dtype: int64

In [67]:
!scrapy crawl GoogleNews

Scrapy 2.5.0 - no active project

Unknown command: crawl

Use "scrapy" to see available commands


In [66]:
ns = NewsSpider()
ns.start_requests('COVID-19')

<generator object NewsSpider.start_requests at 0x7f898475d900>

### Directly search with `requests` 

In [69]:
import requests
from bs4 import BeautifulSoup

query = 'COVID-19'
url = f"https://www.google.com/search?q={query}&tbm=nws&lr=lang_en&hl=en&sort=date"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

In [77]:
soup.findAll("a", {"aria-label": "Next page"})[0]['href']

'/search?q=COVID-19&lr=lang_en&hl=en&ie=UTF-8&tbs=lr:lang_1en&tbm=nws&ei=s7uzYJCUHKyv5NoPrZOFoAI&start=10&sa=N'

In [82]:
url = "https://search.api.cnn.io/content?q=covid-19"
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

In [89]:
res.json()['result'][0]['headline']

"America's Amtrak moment could finally be here"

In [61]:
summaries = []
divs = soup.findAll("div", {"class": "BNeawe vvjwJb AP7Wnd"})
for div in divs:
    summaries.append(div.text)
summaries

['Moderna Warns New Waves of Covid-19 Are Coming',
 'We need to know how Covid-19 emerged so we can stop it happening again',
 'Vietnam detects hybrid of Indian and UK COVID-19 variants',
 'Over 100 staff sue Houston Methodist over COVID-19 vaccine requirement | TheHill',
 'Bars reopening for the first time since the COVID-19 pandemic began',
 'COVID-19 cases and deaths are down but rate of infection is the same as it was in December for unvaccinated',
 'Worried about COVID-19, Navajo Nation ignores CDC, keeps masks and social distancing',
 "Opinion | Covid-19's Impact on America Has Just Begun",
 'Montgomery County has averaged 35 new cases of COVID-19 a day in May',
 'This COVID-19 vaccine shot comes with a free beer',
 'Covid-19 Cases and Hospitalizations Rise in U.K. as Variant From India Takes Hold',
 'OREGON Oregon reports 1 more COVID-19 related death, 376 new cases KTVZ',
 'Coronavirus in DC, Maryland, Virginia: What to Know on May 29',
 'Coronavirus daily news updates, May 29:

In [37]:
import re
from bs4 import BeautifulSoup

In [40]:
soup = BeautifulSoup(result.text)
for title in soup.find_all('span', class_="titletext"):
    print(title.text)